In [ ]:
import gc
import os
import joblib
import random
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# Loading and Preparing Data

In [ ]:
users = pd.read_csv('./Data/users.csv')
cards = pd.read_csv('./Data/cards.csv')
trans = pd.read_csv('./Data/transactions.csv')

In [ ]:
users_needed_cols = ['Current Age', 'Retirement Age', 'Gender', 'Zipcode', 'Per Capita Income - Zipcode', 
                    'Yearly Income - Person', 'Total Debt', 'FICO Score', 'Num Credit Cards']
cards_needed_cols = ['User', 'CARD INDEX', 'Year PIN last Changed', 'Acct Open Date', 'Expires', 'Card Duration', 
                    'Has Chip', 'Cards Issued', 'Credit Limit', 'Visa', 'Mastercard', 'Discover', 'Amex', 'Debit', 
                    'Credit', 'Debit (Prepaid)']
trans_needed_cols = ['User', 'Card', 'Time', 'Time From Now', 'Amount', 'Merchant Name', 'Merchant City', 
                    'Merchant State', 'Zip', 'MCC', 'Errors?', 'Swipe Transaction', 'Online Transaction', 
                    'Chip Transaction', 'Is Fraud?']

In [ ]:
data_prep_models = {}

## Handling Zip Codes

In [ ]:
zip = pd.concat([users.Zipcode, trans.Zip.fillna(0).astype(int)])
zip_encoder = LabelEncoder()
zip_encoder.fit_transform(zip)
users.Zipcode = zip_encoder.transform(users.Zipcode)
trans.Zip = zip_encoder.transform(trans.Zip.fillna(0).astype(int))
data_prep_models['zip_encoder'] = zip_encoder

In [ ]:
users['Zipcode'].min(), users['Zipcode'].max()

(33, 27247)

In [ ]:
trans['Zip'].min(), trans['Zip'].max()

(0, 27321)

In [ ]:
users['Zipcode'] = users['Zipcode'].astype(np.uint16)
trans['Zip'] = trans['Zip'].astype(np.uint16)

## Handling User Ages

In [ ]:
users[['Current Age', 'Retirement Age']].min(), users[['Current Age', 'Retirement Age']].max()

(Current Age       18
 Retirement Age    50
 dtype: int64, Current Age       101
 Retirement Age     79
 dtype: int64)

In [ ]:
users['Current Age'] = users['Current Age'].astype(np.uint8)
users['Retirement Age'] = users['Retirement Age'].astype(np.uint8)

## Handling User Gender

In [ ]:
users['Gender'].value_counts()

Female    1016
Male       984
Name: Gender, dtype: int64

In [ ]:
gender_encoder = LabelEncoder()
gender_encoder.fit_transform(users['Gender'])
users['Gender'] = gender_encoder.transform(users['Gender'])
data_prep_models['gender_encoder'] = gender_encoder

## Handling Money

In [ ]:
users['Per Capita Income - Zipcode'] = users['Per Capita Income - Zipcode'].str.replace('$', '', regex=True).astype(int)
users['Yearly Income - Person'] = users['Yearly Income - Person'].str.replace('$', '', regex=True).astype(int)
users['Total Debt'] = users['Total Debt'].str.replace('$', '', regex=True).astype(int)

## Handling FICO Score

In [ ]:
users['FICO Score'].min(), users['FICO Score'].max()

(480, 850)

In [ ]:
users['FICO Score'] = users['FICO Score'].astype(np.uint16)

## Handling User's Credit Cards Number

In [ ]:
users['Num Credit Cards'] = users['Num Credit Cards'].astype(np.uint8)

## Handling User and Card Index

In [ ]:
cards['User'] = cards['User'].astype(np.uint16)
cards['CARD INDEX'] = cards['CARD INDEX'].astype(np.uint8)

## Handling Card Brand and Type

In [ ]:
cards['Card Brand'].value_counts(), cards['Card Type'].value_counts()

(Mastercard    3209
 Visa          2326
 Amex           402
 Discover       209
 Name: Card Brand, dtype: int64, Debit              3511
 Credit             2057
 Debit (Prepaid)     578
 Name: Card Type, dtype: int64)

In [ ]:
card_brand_type_encoder = OneHotEncoder(categories=[cards['Card Brand'].unique(), cards['Card Type'].unique()], handle_unknown='ignore', dtype=np.uint8)
brand = cards[['Card Brand', 'Card Type']]
card_brand_type_encoder.fit_transform(brand)
brand = pd.DataFrame(card_brand_type_encoder.transform(brand).toarray(), columns=['Visa', 'Mastercard', 'Discover', 'Amex', 'Debit', 'Credit', 'Debit (Prepaid)'])
cards = pd.concat([cards.drop(columns=['Card Brand', 'Card Type']), brand], axis=1)
data_prep_models['card_brand_type_encoder'] = card_brand_type_encoder
card_brand_type_encoder.categories_

[array(['Visa', 'Mastercard', 'Discover', 'Amex'], dtype=object),
 array(['Debit', 'Credit', 'Debit (Prepaid)'], dtype=object)]

## Handle Has Chip

In [ ]:
cards['Has Chip'].value_counts()

YES    5500
NO      646
Name: Has Chip, dtype: int64

In [ ]:
has_chip_encoder = LabelEncoder()
has_chip_encoder.fit_transform(cards['Has Chip'])
cards['Has Chip'] = has_chip_encoder.transform(cards['Has Chip'])
data_prep_models['has_chip_encoder'] = has_chip_encoder

## Handle Cards Issued

In [ ]:
cards['Cards Issued'].min(), cards['Cards Issued'].max()

(1, 3)

In [ ]:
cards['Cards Issued'] = cards['Cards Issued'].astype(np.uint8)

## Handle Card Credit Limit

In [ ]:
cards['Credit Limit'] = cards['Credit Limit'].str.replace('$', '', regex=True).astype(int)

## Handle Cards Dates

In [ ]:
now = dt.datetime.now()
cards['Year PIN last Changed'] += (now.year - cards['Year PIN last Changed']).astype(np.uint16)
cards['Expires'] = ((now - pd.to_datetime(cards['Expires'], infer_datetime_format=True)) / np.timedelta64(1, 'M')).astype(int)
cards['Acct Open Date'] = ((now - pd.to_datetime(cards['Acct Open Date'], infer_datetime_format=True)) / np.timedelta64(1, 'M')).astype(int)
cards['Card Duration'] = cards['Expires'] - cards['Acct Open Date']

## Transaction User and Card

In [ ]:
trans.sort_values(by=['User', 'Card', 'Year', 'Month', 'Day', 'Time'], inplace=True)
trans['User'] = trans['User'].astype(np.uint16)
trans['Card'] = trans['Card'].astype(np.uint8)

## Handling Date and Time

In [ ]:
trans['Time From Now'] = (now - pd.to_datetime(dict(year=trans['Year'], month=trans['Month'], day=trans['Day']))).dt.days
trans['Time'] = trans['Time'].str.replace(':[0-9]+', '', regex=True).astype(np.uint8)

## Handling Amount

In [ ]:
trans['Amount'] = trans['Amount'].str.replace('$', '', regex=True).astype(float).apply(lambda x: np.log(max(1.0, x)))

## Handling Use Chip

In [ ]:
use_chip_encoder = OneHotEncoder(categories=[list(trans['Use Chip'].unique())], handle_unknown='ignore', dtype=np.uint8)
use_chip_encoder.fit_transform(trans[['Use Chip']])
use_chip = pd.DataFrame(use_chip_encoder.transform(trans[['Use Chip']]).toarray(), columns=['Swipe Transaction', 'Online Transaction', 'Chip Transaction'])
trans = pd.concat([trans.drop(columns=['Use Chip']), use_chip], axis=1)
data_prep_models['use_chip_encoder'] = use_chip_encoder
use_chip_encoder.categories_

[array(['Swipe Transaction', 'Online Transaction', 'Chip Transaction'],
       dtype=object)]

## Handling Merchant City, State, Name and MCC

In [ ]:
merchant_name_encoder = LabelEncoder()
merchant_name_encoder.fit(trans['Merchant Name'])
trans['Merchant Name'] = merchant_name_encoder.transform(trans['Merchant Name'])
data_prep_models['merchant_name_encoder'] = merchant_name_encoder

In [ ]:
trans['Merchant Name'].min(), trans['Merchant Name'].max()

(0, 100342)

In [ ]:
trans['Merchant Name'] = trans['Merchant Name'].astype(np.uint32)

In [ ]:
merchant_city_encoder = LabelEncoder()
merchant_city_encoder.fit(trans['Merchant City'])
trans['Merchant City'] = merchant_city_encoder.transform(trans['Merchant City'])
data_prep_models['merchant_city_encoder'] = merchant_city_encoder

In [ ]:
trans['Merchant City'].min(), trans['Merchant City'].max()

(0, 13428)

In [ ]:
trans['Merchant City'] = trans['Merchant City'].astype(np.uint16)

In [ ]:
trans['Merchant State'].fillna('Unknown', inplace=True)
merchant_state_encoder = LabelEncoder()
merchant_state_encoder.fit(trans['Merchant State'])
trans['Merchant State'] = merchant_state_encoder.transform(trans['Merchant State'])
data_prep_models['merchant_state_encoder'] = merchant_state_encoder

In [ ]:
trans['Merchant State'].min(), trans['Merchant State'].max()

(0, 223)

In [ ]:
trans['Merchant State'] = trans['Merchant State'].astype(np.uint8)

In [ ]:
mcc_encoder = LabelEncoder()
mcc_encoder.fit(trans['MCC'])
trans['MCC'] = mcc_encoder.transform(trans['MCC'])
data_prep_models['mcc_encoder'] = mcc_encoder

In [ ]:
trans['MCC'].min(), trans['MCC'].max()

(0, 108)

In [ ]:
trans['Merchant State'] = trans['Merchant State'].astype(np.uint8)

## Handling Errors

In [ ]:
trans['Errors?'].fillna('No Errors', inplace=True)
errors_encoder = LabelEncoder()
errors_encoder.fit(trans['Errors?'])
trans['Errors?'] = errors_encoder.transform(trans['Errors?'])
data_prep_models['errors_encoder'] = errors_encoder

In [ ]:
trans['Errors?'].min(), trans['Errors?'].max()

(0, 23)

In [ ]:
trans['Errors?'] = trans['Errors?'].astype(np.uint8)

In [ ]:
trans['Is Fraud?'] = trans['Is Fraud?'].replace({'No': 0, 'Yes': 1}).astype(np.uint8)

## Saving Data Prep Models

In [ ]:
joblib.dump(data_prep_models, './Models/DataPrepModels.pkl')

['./Models/DataPrepModels.pkl']

## Saving Final Data

In [ ]:
dtypes = {}
dtypes['users'] = users[users_needed_cols].dtypes.to_dict()
dtypes['cards'] = cards[cards_needed_cols].dtypes.to_dict()
dtypes['trans'] = trans[trans_needed_cols].dtypes.to_dict()
joblib.dump(dtypes, './Data/FinalData/types.pkl')

['./Data/FinalData/types.pkl']

In [ ]:
users[users_needed_cols].to_csv('./Data/FinalData/users.csv', index=False)
cards[cards_needed_cols].to_csv('./Data/FinalData/cards.csv', index=False)
trans[trans_needed_cols].to_csv('./Data/FinalData/trans.csv', index=False)

In [ ]:
!zip -r final_data.zip /content/Data
!zip -r final_models.zip /content/Models

  adding: content/Data/ (stored 0%)
  adding: content/Data/.ipynb_checkpoints/ (stored 0%)
  adding: content/Data/FinalData/ (stored 0%)
  adding: content/Data/FinalData/trans.csv (deflated 80%)
  adding: content/Data/FinalData/users.csv (deflated 55%)
  adding: content/Data/FinalData/cards.csv (deflated 74%)
  adding: content/Data/FinalData/types.pkl (deflated 43%)
  adding: content/Models/ (stored 0%)
  adding: content/Models/DataPrepModels.pkl (deflated 28%)


In [ ]:
!cp /content/final_models.zip /content/drive/MyDrive/IBM_Dataset/
!cp /content/final_data.zip /content/drive/MyDrive/IBM_Dataset/

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/IBM_Dataset/final_data.zip /content/
!cp /content/drive/MyDrive/IBM_Dataset/final_models.zip /content/

In [ ]:
!unzip final_data.zip
!unzip final_models.zip
!mv /content/content/Data Data
!mv /content/content/Models Models
!rm -r /content/content
!rm final_data.zip
!rm final_models.zip

In [ ]:
types = joblib.load('./Data/FinalData/types.pkl')

users_needed_cols = ['Current Age', 'Retirement Age', 'Gender', 'Zipcode', 'Per Capita Income - Zipcode', 
                    'Yearly Income - Person', 'Total Debt', 'FICO Score', 'Num Credit Cards']
cards_needed_cols = ['User', 'CARD INDEX', 'Year PIN last Changed', 'Acct Open Date', 'Expires', 'Card Duration', 
                    'Has Chip', 'Cards Issued', 'Credit Limit', 'Visa', 'Mastercard', 'Discover', 'Amex', 'Debit', 
                    'Credit', 'Debit (Prepaid)']
trans_needed_cols = ['User', 'Card', 'Time', 'Time From Now', 'Amount', 'Merchant Name', 'Merchant City', 
                    'Merchant State', 'Zip', 'MCC', 'Errors?', 'Swipe Transaction', 'Online Transaction', 
                    'Chip Transaction', 'Is Fraud?']

users = pd.read_csv('./Data/FinalData/users.csv', usecols=users_needed_cols, dtype=types['users'])
cards = pd.read_csv('./Data/FinalData/cards.csv', usecols=cards_needed_cols, dtype=types['cards'])
if os.path.exists('/content/drive/MyDrive/IBM_Dataset/trans_df.pkl'):
  trans_df = joblib.load('/content/drive/MyDrive/IBM_Dataset/trans_df.pkl')
  targets = joblib.load('/content/drive/MyDrive/IBM_Dataset/targets.pkl')
else:
  trans = pd.read_csv('./Data/FinalData/trans.csv', usecols=trans_needed_cols, dtype=types['trans'])

In [ ]:
if not os.path.exists('/content/drive/MyDrive/IBM_Dataset/trans_df.pkl')
  ids = {1869, 1897, 1594, 1727, 1627, 1527, 1474, 1418, 1307, 1807, 1021,
          1997,  761,  553,  752,  288,  533,  151,  129,  165,  529,   16,
            3,  802,  521,   42,  440, 1898, 1397,  786, 1893,   91, 1034,
          1428,  453, 1146,  209, 1778,  398, 1534, 1622,   22,  291,  952,
          1192, 1564,  539, 1328,   89,  359, 1305,  330,  420,  644,  836,
          256,  688,  234,  574, 1188,  953, 1204, 1508, 1420, 1524,  139,
          699, 1083, 1349, 1343, 1091,   62,  748,  794, 1102, 1103,  744,
          1393,  473,   97, 1291, 1896, 1584,  182,  128,  540, 1725,  943,
          899,  490, 1601,  494, 1589,  428,  938,  853, 1224,  515,  883,
          823,  692, 1823, 1417, 1887, 1777, 1938, 1361, 1306, 1319,   53,
          1755, 1759, 1977,  966,  657,  711, 1115, 1895,  876, 1882,  718,
          1990, 1017,  150,  720,  751, 1478,  463,  271, 1180, 1693,  934,
          1266, 1656, 1649,  418,  864, 1169,  672,  979,  215, 1533,  595,
          901,  559,  714,  569, 1657,  520, 1962, 1934, 1038, 1107, 1006,
          1841, 1809, 1802, 1567, 1773,  268,  299, 1736,  318, 1790,  782,
          462,  907,  896, 1597,  452,  509, 1258,  729,   75,  881, 1913,
          1877,  365,  188,   61,   77,  809,  503, 1158, 1735,  662, 1764,
          831, 1223, 1178, 1470,  776,   49,   55, 1945, 1929,  621,  647,
          1820,  726, 1918, 1519, 1908,  544,  180, 1522, 1827, 1444, 1734,
            0, 1166, 1008, 1106, 1049,  827, 1052, 1173,  790,  885,  920,
          1236,  929}
  trans = trans[trans['User'].isin(ids)]

# Prepare Transactions Windows

In [ ]:
stride = 5
seq_len = 10

In [ ]:
trans.reset_index(inplace=True)

In [ ]:
if not os.path.exists('/content/drive/MyDrive/IBM_Dataset/trans_df.pkl')  
  trans_df = []
  targets = []
  trans_feats = [
      'Time', 'Time From Now', 'Amount', 'Merchant Name', 'Merchant City', 'Merchant State', 
      'Zip', 'MCC', 'Errors?', 'Swipe Transaction', 'Online Transaction', 'Chip Transaction'
  ]
  user_card_dict = trans.groupby('User')['Card'].unique().to_dict()
  trans.set_index(['User', 'Card'], inplace=True)

  for user_id in tqdm(user_card_dict.keys()):
      for card_id in user_card_dict[user_id]:
          t_df = trans.loc[user_id, card_id]
          for i in range(0, t_df.shape[0], stride):
              wind_df = t_df[i:i+seq_len]
              trans_df.append((user_id, card_id, wind_df[trans_feats]))
              targets.append(wind_df['Is Fraud?'].iloc[-1])
  joblib.dump(trans_df, '/content/drive/MyDrive/IBM_Dataset/trans_df.pkl')
  joblib.dump(targets, '/content/drive/MyDrive/IBM_Dataset/targets.pkl')

In [ ]:
x_train_val, x_test, y_train_val, y_test = train_test_split(trans_df, targets, test_size=0.3, random_state=42, stratify=targets)
x_train, x_val, y_train, y_val = train_test_split(trans_df, targets, test_size=0.1, random_state=42, stratify=targets)

# Model Parameters

In [ ]:
batch_size = 8
n_step = 10
users_feats = 9
cards_feats = 14
trans_feats = 12
n_feats = users_feats + cards_feats + trans_feats
n_feats

35

# Data Loader

In [ ]:
def get_batch(user_df: pd.DataFrame, card_df: pd.DataFrame, trans_df: list, target: list, batch_size: int, conv_input=False):
    card_df.set_index(['User', 'CARD INDEX'], inplace=True)
    while True:
        idx = np.random.choice(range(len(trans_df)), size=len(trans_df), replace=False)
        for i in range(0, len(idx), batch_size):
            t_df = [trans_df[idx[j]] for j in range(i, i + batch_size)]
            X = np.zeros((len(t_df), n_step, n_feats))
            Y = np.array([targets[idx[j]] for j in range(i, i + batch_size)])

            for it, (user_id, card_id, wind_df) in enumerate(t_df):
                if wind_df.shape[0] < seq_len:
                  continue
                user = user_df.iloc[[user_id]]
                card = card_df.loc[user_id, card_id].to_frame().T
                # print(user.shape, card.shape, wind_df.reset_index(drop=True).shape)
                x = pd.concat([user.reset_index(drop=True), card.reset_index(drop=True), wind_df[trans_feats].reset_index(drop=True)], axis=1).ffill()

                # print(x)
                # print(x.columns)
                X[it] = x
            if conv_input:
                X = np.transpose(X, axis=[0, 2, 1])
            yield X, Y
cards.reset_index(inplace=True)

# Training Plots

In [ ]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(4)
    fig.tight_layout(pad=3)

    axs[0].plot(history.history["loss"], label="train loss")
    axs[0].plot(history.history["val_loss"], label="val loss")
    axs[0].set_ylabel("loss")
    axs[0].set_xlabel("Epoch")
    axs[0].legend(loc="upper right")
    axs[0].set_title("Loss")

    axs[1].plot(history.history["auc"], label="train AUC")
    axs[1].plot(history.history["val_auc"], label="val AUC")
    axs[1].set_ylabel("AUC")
    axs[1].legend(loc="lower right")
    axs[1].set_title("AUC")

    axs[2].plot(history.history["precision"], label="train precision")
    axs[2].plot(history.history["val_precision"], label="val precision")
    axs[2].set_ylabel("Precision")
    axs[2].legend(loc="lower right")
    axs[2].set_title("Precision")

    axs[3].plot(history.history["recall"], label="train recall")
    axs[3].plot(history.history["val_recall"], label="val recall")
    axs[3].set_ylabel("Recall")
    axs[3].legend(loc="lower right")
    axs[3].set_title("Recall")


# Models

## LSTM

In [ ]:
def LstmModel():
  model = keras.Sequential()
  
  model.add(layers.LSTM(units=64, kernel_regularizer='l2', return_sequences=True, time_major=False, input_shape=(n_step, n_feats)))
  model.add(layers.LSTM(units=32, kernel_regularizer='l2', return_sequences=True, time_major=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(units=16, kernel_regularizer='l2', return_sequences=False, time_major=False))
  model.add(layers.Dense(16))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

## GRU

In [ ]:
def GruModel():
  model = keras.Sequential()
  
  model.add(layers.GRU(units=64, kernel_regularizer='l2', return_sequences=True, time_major=False, input_shape=(n_step, n_feats)))
  model.add(layers.GRU(units=32, kernel_regularizer='l2', return_sequences=True, time_major=False))
  model.add(layers.BatchNormalization())
  model.add(layers.GRU(units=16, kernel_regularizer='l2', return_sequences=False, time_major=False))
  model.add(layers.Dense(16))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

## Conv1D

In [ ]:
def ConvModel():
  model = keras.Sequential()
  
  model.add(layers.Conv1D(64, 3, activation='relu', kernel_regularizer='l2', input_shape=(n_feats, n_step)))
  model.add(layers.AveragePooling1D(pool_size=2, strides=2))
  model.add(layers.Conv1D(128, 7, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(512, 5, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(1024, 5, activation='relu', kernel_regularizer='l2'))
  model.add(layers.AveragePooling1D(pool_size=2))
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(128))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

## Conv-LSTM

In [ ]:
def ConvLstmModel():
  model = keras.Sequential()

  model.add(layers.Conv1D(64, 3, activation='relu', kernel_regularizer='l2', input_shape=(n_feats, n_step)))
  model.add(layers.AveragePooling1D(pool_size=2, strides=2))
  model.add(layers.Conv1D(128, 3, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(512, 5, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(1024, 5, activation='relu', kernel_regularizer='l2'))

  model.add(layers.Dense(256))
  model.add(layers.Dropout(0.3))

  model.add(layers.LSTM(64, kernel_regularizer='l2',  return_sequences=True))
  model.add(layers.LSTM(64, kernel_regularizer='l2',  return_sequences=False))
  
  model.add(layers.Dense(256))
  model.add(layers.Dense(128))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

## Auto-Encoder

### Conv1D

In [ ]:
def ConvAE():
    input = layers.Input(shape=(n_feats, n_step))
    x = layers.Conv1D(64, 3, strides=2, activation='relu', kernel_regularizer='l2')(input)
    x = layers.Conv1D(128, 5, strides=2, activation='relu', kernel_regularizer='l2')(x)
    x = layers.AveragePooling1D(4)(x)
    y = layers.Conv1DTranspose(128, 1, strides=7, activation='relu', kernel_regularizer='l2')(x)
    y = layers.Conv1DTranspose(64, 5, strides=2, activation='relu', kernel_regularizer='l2')(y)
    y = layers.Conv1DTranspose(10, 3, strides=2, activation='relu', kernel_regularizer='l2')(y)
    encoder = Model(inputs=input, outputs=x)
    ae = Model(inputs=input, outputs=y)
    return ae, encoder

### LSTM

In [ ]:
def LstmAE():
    input = layers.Input(shape=(n_step, n_feats))
    x = layers.LSTM(units=64, kernel_regularizer='l2', return_sequences=True)(input)
    x = layers.LSTM(units=32, kernel_regularizer='l2', return_sequences=False)(x)
    y = layers.RepeatVector(n_step)(x)
    y = layers.LSTM(units=32, kernel_regularizer='l2', return_sequences=True)(y)
    y = layers.LSTM(units=64, kernel_regularizer='l2', return_sequences=True)(y)
    output = layers.TimeDistributed(layers.Dense(n_feats))(y)
    encoder = Model(inputs=input, outputs=x)
    ae = Model(inputs=input, outputs=output)
    return ae, encoder

# Model Definition

In [ ]:
def get_model(model_name):
    model = None
    if model_name == 'LSTM':
        model = LstmModel()
    elif model_name == 'GRU':
        model = GruModel()
    elif model_name == 'CONV':
        model = ConvModel()
    elif model_name == 'CONVLSTM':
        model = ConvLstmModel()
    elif model_name == 'CONVAE':
        model = ConvAE()
    elif model_name == 'LSTMAE':
        model = LstmAE()
    return model

# Training

In [ ]:
lr = 0.001
epochs = 20
weight_decay = 0.0
checkpoint_dir = '/content/drive/MyDrive/IBM_Dataset/checkpoints'

In [ ]:
model_name = 'LSTM'
model = get_model(model_name)
class_weights = compute_class_weight(class_weight='balanced', classes=[0, 1], y=targets)
class_weights = {0: class_weights[0], 1: class_weights[1]}

In [ ]:
train_loader = get_batch(users, cards, x_train, y_train, batch_size, True if model_name in {'CONV', 'CONVLSTM', 'CONVAE'} else False)
val_loader = get_batch(users, cards, x_val, y_val, batch_size, True if model_name in {'CONV', 'CONVLSTM', 'CONVAE'} else False)
test_loader = get_batch(users, cards, x_test, y_test, batch_size, True if model_name in {'CONV', 'CONVLSTM', 'CONVAE'} else False)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr, weight_decay=weight_decay), 
    loss='binary_crossentropy',
    metrics=[keras.metrics.AUC()],
    weighted_metrics=[keras.metrics.Precision(), keras.metrics.Recall()]
)
train_details = model.fit(
    x=train_loader,
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_val) // batch_size,
    epochs=epochs,
    validation_data=val_loader,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, monitor='val_loss'),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(checkpoint_dir, model_name), monitor='val_loss', save_best_only=True)
    ],
    class_weight=class_weights
)

Epoch 1/20
  995/88058 [..............................] - ETA: 1:43:28 - loss: 1.5212 - auc_2: 0.4885 - precision_2: 0.5269 - recall_2: 0.2000

KeyboardInterrupt: ignored